In [1]:
import os
import numpy as np
import pandas as pd

from pandas.errors import EmptyDataError

from sqlalchemy import create_engine
from sqlalchemy import text

from tqdm import tqdm

In [2]:
#Setup Database
#Setup connection parameters
username = 'plankton'
password = 'piscodisco'
host = 'localhost'  # or the IP address of your database server
port = '5432'       # default port for PostgreSQL
database = 'pisco_crops_db'

# Create an engine that connects to the PostgreSQL server
engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{database}')


In [3]:
# Fetch all table names in the database
query_tables = "SELECT tablename FROM pg_tables WHERE schemaname='public'"
with engine.connect() as conn:
    table_names = pd.read_sql_query(text(query_tables), conn)['tablename'].tolist()
print(table_names)
print(len(table_names),'entries found')

['M181-267-1_CTD-070_00deg00S-035deg00W_20220516-0335', 'M181-200-1_CTD-055_00deg00S-023deg00W_20220511-0023', 'M181-227-1_CTD-060_00deg00S-027deg00W_20220512-1748', 'M181-126-1_CTD-040_00deg00S-010deg00W_20220505-1401', 'M181-107-1_CTD-036_00deg00S-007deg00W_20220504-0756', 'M181-175-1_CTD-050_00deg00S-019deg00W_20220509-0543', 'M181-140-1_CTD-043_00deg00S-013deg00W_20220506-1822', 'M181-160-1_CTD-047_00deg00S-016deg00W_20220508-0038', 'M181-285-1_CTD-075_00deg00S-039deg00W_20220517-2024', 'M181-295-1_CTD-080_00deg00S-041deg00W_20220518-2053', 'M181-245-1_CTD-065_00deg00S-031deg00W_20220514-0907', 'M181-220-1_CTD-059_00deg00S-026deg00W_20220512-0920', 'M181-216-1_CTD-058_00deg00S-025deg00W_20220511-2352']
13 entries found


In [15]:
def analyze_table(engine, table_name):
    """Analyze a single table from the database"""
    query = f'SELECT * FROM "{table_name}"'
    df = pd.read_sql_query(query, engine)
    
    analysis = {
        'Table Name': table_name,
        'Total Rows': len(df),
        'Total Columns': len(df.columns),
        'Unique obj_ids': len(df['obj_id'].unique()),
        'Duplicate obj_ids': df.duplicated(subset='obj_id').sum(),
        'Missing Values': df.isnull().sum().sum(),
        'Memory Usage': f"{df.memory_usage().sum() / 1024**2:.2f} MB"
    }
    
    return analysis

column_mapping = {
    'area_exc': 'object_area_exc',
    'area_rprops': 'object_area_rprops',
    '%area': 'object_%area',
    'major': 'object_major_axis_len',
    'minor': 'object_minor_axis_len', 
    'centroid_y': 'object_centroid_y',
    'centroid_x': 'object_centroid_x',
    'convex_area': 'object_convex_area',
    'min_intensity': 'object_min_intensity',
    'max_intensity': 'object_max_intensity',
    'mean_intensity': 'object_mean_intensity',
    'intden': 'object_int_density',
    'perim.': 'object_perimeter',
    'elongation': 'object_elongation',
    'range': 'object_range',
    'perimareaexc': 'object_perim_area_excl',
    'perimmajor': 'object_perim_major',
    'circex': 'object_circularity_area_excl',
    'angle': 'object_angle',
    'bounding_box_area': 'object_boundbox_area',
    'eccentricity': 'object_eccentricity',
    'equivalent_diameter': 'object_equivalent_diameter',
    'euler_number': 'object_euler_nr',
    'extent': 'object_extent',
    'local_centroid_col': 'object_local_centroid_col',
    'local_centroid_row': 'object_local_centroid_row',
    'solidity': 'object_solidity',
    'circ.': 'object_circularity'
}
# Analyze all tables
results = []
for table_name in table_names:
    results.append(analyze_table(engine, table_name))
    # To rename columns in your first DataFrame:
    query = f'SELECT * FROM "{table_name}"'
    df = pd.read_sql_query(query, engine)
    df = df.rename(columns=column_mapping)
    df.to_sql(table_name, engine, if_exists='replace', index=False)

# Create a DataFrame with the results
analysis_df = pd.DataFrame(results)
print("Database Analysis:")
print("=" * 80)
print(analysis_df.to_string(index=False))

Database Analysis:
                                         Table Name  Total Rows  Total Columns  Unique obj_ids  Duplicate obj_ids  Missing Values Memory Usage
M181-267-1_CTD-070_00deg00S-035deg00W_20220516-0335       78602             64           78602                  0               0     37.33 MB
M181-200-1_CTD-055_00deg00S-023deg00W_20220511-0023      117528             64          117511                 17               0     55.82 MB
M181-227-1_CTD-060_00deg00S-027deg00W_20220512-1748      112421             64          112349                 72               0     53.39 MB
M181-126-1_CTD-040_00deg00S-010deg00W_20220505-1401      117343             64          117337                  6               0     55.73 MB
M181-107-1_CTD-036_00deg00S-007deg00W_20220504-0756      196612             64          196602                 10               0     93.38 MB
M181-175-1_CTD-050_00deg00S-019deg00W_20220509-0543      105198             64          105195                  3          

In [8]:
for table_name in table_names:
    query = f'SELECT * FROM "{table_name}"'
    df = pd.read_sql_query(query, engine)
    print(table_name, len(df['obj_id']))
    print(len(df.index), len(df.columns))
    #print(df.columns)
    # assuming df is your DataFrame and 'col' is the column you want to count duplicates in
    duplicates = df.duplicated(subset='obj_id')

    # count the duplicates
    num_duplicates = duplicates.sum()

    print(num_duplicates)
    #break

M181-267-1_CTD-070_00deg00S-035deg00W_20220516-0335 78602
78602 64
0
M181-200-1_CTD-055_00deg00S-023deg00W_20220511-0023 117528
117528 64
17
M181-227-1_CTD-060_00deg00S-027deg00W_20220512-1748 112421
112421 64
72
M181-126-1_CTD-040_00deg00S-010deg00W_20220505-1401 117343
117343 64
6
M181-107-1_CTD-036_00deg00S-007deg00W_20220504-0756 196612
196612 64
10
M181-175-1_CTD-050_00deg00S-019deg00W_20220509-0543 105198
105198 64
3
M181-140-1_CTD-043_00deg00S-013deg00W_20220506-1822 71465
71465 64
2
M181-160-1_CTD-047_00deg00S-016deg00W_20220508-0038 104971
104971 64
4
M181-285-1_CTD-075_00deg00S-039deg00W_20220517-2024 46641
46641 64
18
M181-295-1_CTD-080_00deg00S-041deg00W_20220518-2053 92737
92737 64
3
M181-245-1_CTD-065_00deg00S-031deg00W_20220514-0907 130670
130670 64
0
M181-220-1_CTD-059_00deg00S-026deg00W_20220512-0920 98464
98464 66
3


In [18]:
query = f'SELECT * FROM "M181-220-1_CTD-059_00deg00S-026deg00W_20220512-0920"'
df = pd.read_sql_query(query, engine)

print(len(df['date-time'].drop_duplicates()))

print(df.columns)
len(df.columns)

#df['overview_path'].iloc[0]

10409
Index(['date-time', 'pressure [dbar]', 'depth [m]', 'temperature', 'index',
       'img_id', 'filename', 'mean_raw', 'std_raw', 'mean', 'std', 'area', 'x',
       'y', 'w', 'h', 'saved', 'object_bound_box_w', 'object_bound_box_h',
       'bound_box_x', 'bound_box_y', 'object_circularity', 'object_area_exc',
       'object_area_rprops', 'object_%area', 'object_major_axis_len',
       'object_minor_axis_len', 'object_centroid_y', 'object_centroid_x',
       'object_convex_area', 'object_min_intensity', 'object_max_intensity',
       'object_mean_intensity', 'object_int_density', 'object_perimeter',
       'object_elongation', 'object_range', 'object_perim_area_excl',
       'object_perim_major', 'object_circularity_area_excl', 'object_angle',
       'object_boundbox_area', 'object_eccentricity',
       'object_equivalent_diameter', 'object_euler_nr', 'object_extent',
       'object_local_centroid_col', 'object_local_centroid_row',
       'object_solidity', 'full_path', 'esd', 'over

66

In [19]:
# To check which columns are unique to each DataFrame:
#df1_columns = set(df.columns)
df2_columns = set(df.columns)

print("Columns only in first DataFrame:", df1_columns - df2_columns)
print("Columns only in second DataFrame:", df2_columns - df1_columns)

Columns only in first DataFrame: {'width', 'umap_y', 'height', 'by', 'umap_x', 'bx'}
Columns only in second DataFrame: {'bound_box_x', 'std_raw', 'object_bound_box_w', 'std', 'bound_box_y', 'object_bound_box_h', 'mean_raw', 'mean'}


In [63]:
df_unique = df[['date-time', 'pressure [dbar]', 'depth [m]', 'img_id','temperature','overview_path','interpolated_s','interpolated_t','interpolated_o','interpolated_chl','interpolated_z_factor','restart','relock','TAG_event']].drop_duplicates()
#df_unique = df[['date-time']].drop_duplicates()
#df_count = df.groupby('date-time').size().reset_index(name='count')
#df_unique = df_unique.merge(df_count, on='date-time', how='left')
#df_unique = df_unique.sort_values('pressure [dbar]')

In [45]:
len(df_unique)

7759

In [15]:
# Find the index of the minimum value in 'esd' column
index = df['esd'].idxmin()

# Use .iloc[] to get the corresponding value in the 'area' column for the minimum index
min_value_area = df.iloc[index]['area']

print('Minimum value in "esd" column:', df.iloc[index]['esd'])
print('Corresponding value in "area" column:', min_value_area)

Minimum value in "esd" column: 609.71
Corresponding value in "area" column: 400.5


In [14]:
def modify_full_path(path):
    dirname, base_name = os.path.split(path)
    base_parts = base_name.split('_')
    new_base_name = '_'.join(base_parts[:-1]) + '.png'
    return os.path.join(dirname.replace('Crops', 'Images'), new_base_name)

df['overview_path'] = df['full_path'].apply(modify_full_path)
print(df.head())

           date-time  pressure [dbar]  depth [m]  temperature  index  img_id  \
0  20220505-14191884            -0.32     -0.318        28.51     17       1   
1  20220505-14191884            -0.32     -0.318        28.51     73       1   
2  20220505-14191884            -0.32     -0.318        28.51     93       1   
3  20220505-14191884            -0.32     -0.318        28.51    102       1   
4  20220505-14191884            -0.32     -0.318        28.51    106       1   

                                      filename     area       x       y  ...  \
0   20220505-14191884_000.968bar_28.51C_17.png   1556.0   432.5  2514.5  ...   
1   20220505-14191884_000.968bar_28.51C_73.png    547.5  2166.0  2325.0  ...   
2   20220505-14191884_000.968bar_28.51C_93.png    738.5  2095.0  2286.5  ...   
3  20220505-14191884_000.968bar_28.51C_102.png    501.5   663.0  2276.0  ...   
4  20220505-14191884_000.968bar_28.51C_106.png  14396.5   414.0  2325.0  ...   

        angle  bounding_box_area  ecce

In [58]:
restart_rows = df_unique[df_unique['relock'] == True]
print(len(restart_rows), restart_rows)

64                date-time  pressure [dbar]  depth [m]  img_id  temperature  \
11770  20220518-21073407             3.27      3.250    1102        28.88   
11792  20220518-21073432             3.29      3.270    1103        28.88   
11810  20220518-21073457             3.29      3.270    1104        28.88   
11826  20220518-21073482             3.27      3.250    1105        28.90   
80703  20220518-21162510           413.42    410.736    3226         9.47   
...                  ...              ...        ...     ...          ...   
90801  20220518-22025680          3433.15   3386.777   14391         2.47   
90808  20220518-22025705          3433.33   3386.953   14392         2.47   
90816  20220518-22025730          3433.56   3387.182   14393         2.47   
90835  20220518-22025755          3433.76   3387.380   14394         2.47   
90857  20220518-22025780          3434.01   3387.620   14395         2.47   

                                           overview_path  interpolated_s

In [17]:
from tqdm import tqdm
for table_name in tqdm(table_names):
    query = f'SELECT * FROM "{table_name}"'
    df = pd.read_sql_query(query, engine)
    df['overview_path'] = df['full_path'].apply(modify_full_path)
    df.to_sql(table_name, engine, if_exists='replace', index=False)

100%|██████████| 11/11 [05:04<00:00, 27.65s/it]


### Add log information 

In [190]:
# # Function to reformat timestamp
# def reformat_timestamp(timestamp):
#     # Format: YYYYMMDD_HHh_MMm_SSs to YYYYMMDD-HHMMSS
#     formatted_timestamp = re.sub(r'(\d{4})(\d{2})(\d{2})_(\d{2})h_(\d{2})m_(\d{2})s', r'\1\2\3-\4\5\6', timestamp)
#     return formatted_timestamp

# def parse_line(line, row):
#     if line.startswith("b'TT"):
#         temp_values = line[2:].rstrip("'").split('_')
#         row['TT']= float(temp_values[1])
#         row['T1']= float(temp_values[3])
#         row['T2']= float(temp_values[5])
#         row['TH']= float(temp_values[7])

#     # # elif line.startswith("b'Heat"):
#     # #     return {}
#     # # elif line.startswith("b'Cool"):
#     # #     return {}
#     # else:
#     #     return {}

# def create_log_df(file_path):

#     # Read the log file
#     with open(file_path, 'r') as f:
#         lines = f.readlines()

#     data = []
#     temp_data = {}

#     # Iterate through lines
#     for line in lines:
#         line = line.rstrip('\n')
#         # Check if line is a timestamp
#         if "h" in line and "m" in line and "s" in line:
#             if temp_data:
#                 # If current timestamp already exists in data, update the existing dictionary
#                 if any(d['timestamp'] == temp_data['timestamp'] for d in data):
#                     existing_data = [d for d in data if d['timestamp'] == temp_data['timestamp']][0]
#                     existing_data.update(temp_data)
#                 else:
#                     data.append(temp_data)
#                 temp_data = {}
#             temp_data['timestamp'] = line
#         else:
#             # Parse line according to message type
#             if line.startswith("b'TT"):
#                 temp_values = line[2:].rstrip("'").split('_')
#                 temp_data['TT'] = float(temp_values[1])
#                 temp_data['T1'] = float(temp_values[3])
#                 temp_data['T2'] = float(temp_values[5])
#                 temp_data['TH'] = float(temp_values[7])
#             elif line.startswith('Restart Tag'):
#                 temp_data['restart'] = True
#             elif line == 'Relock':
#                 temp_data['relock'] = True

#     # If there is data waiting after the last line, add it
#     if temp_data:
#         if any(d['timestamp'] == temp_data['timestamp'] for d in data):
#             existing_data = [d for d in data if d['timestamp'] == temp_data['timestamp']][0]
#             existing_data.update(temp_data)
#         else:
#             data.append(temp_data)

#     # Create a dataframe from the data list
#     df = pd.DataFrame(data)

#     # Convert timestamps to datetime and set it as index
#     df['timestamp'] = pd.to_datetime(df['timestamp'], format="%Y%m%d_%Hh_%Mm_%Ss")
#     df.set_index('timestamp', inplace=True)

#     # Replace NaN values in 'relock' and 'restart' columns with False
#     df[['restart', 'relock']] = df[['restart', 'relock']].fillna(False)
#     cols = ['TT', 'T1', 'T2', 'TH']  # list of your column names
#     for col in cols:
#         df[col] = df[col].interpolate()
    
#     return df

In [59]:
## new create log function
def create_log_df(file_path):
    # Read the log file
    with open(file_path, 'r') as f:
        lines = f.readlines()

    data = []
    temp_data = {}
    indicator = 0
    counter = 1

    # Iterate through lines
    for line in lines:
        line = line.rstrip('\n')
        #print(line)
        # Check if line is a timestamp
        if "h" in line and "m" in line and "s" in line:
            if temp_data and 'timestamp' in temp_data:
                # If current timestamp already exists in data, update the existing dictionary
                if any(d['timestamp'] == temp_data['timestamp'] for d in data):
                    existing_data = [d for d in data if d['timestamp'] == temp_data['timestamp']][0]
                    existing_data.update(temp_data)
                else:
                    data.append(temp_data)
                temp_data = {}
            temp_data['timestamp'] = line
        else:
            # Parse line according to message type
            if line.startswith("b'TT"):
                temp_values = line[2:].rstrip("'").split('_')
                temp_data['TT'] = float(temp_values[1])
                temp_data['T1'] = float(temp_values[3])
                temp_data['T2'] = float(temp_values[5])
                temp_data['TH'] = float(temp_values[7])
            elif line.startswith('Restart Tag'):
                temp_data['restart'] = True
                indicator = 0
            elif line == 'Relock':
                temp_data['relock'] = True
                indicator = counter
                counter += 1
            temp_data['TAG_event'] = indicator

    # If there is data waiting after the last line, add it
    if temp_data:
        if any(d['timestamp'] == temp_data['timestamp'] for d in data):
            existing_data = [d for d in data if d['timestamp'] == temp_data['timestamp']][0]
            existing_data.update(temp_data)
        else:
            data.append(temp_data)

    # Create a dataframe from the data list
    df = pd.DataFrame(data)
    df['timestamp'] = pd.to_datetime(df['timestamp'], format="%Y%m%d_%Hh_%Mm_%Ss")
    df.set_index('timestamp', inplace=True)

    # Replace NaN values in 'relock' and 'restart' columns with False
    df[['restart', 'relock']] = df[['restart', 'relock']].fillna(False)
    cols = ['TT', 'T1', 'T2', 'TH']  # list of your column names
    for col in cols:
        df[col] = df[col].interpolate()
    
    return df


In [60]:
import re
import pandas as pd
import os
import datetime

directory = '/media/plankton/30781fe1-cea5-4503-ae00-1986beb935d2/Segmentation_results/M181/Templog/'

# find the right logfile for each profile
for profile in table_names:
    timestamp = profile[-13:]
    # Convert timestamp to datetime object
    date_time_obj = datetime.datetime.strptime(timestamp, '%Y%m%d-%H%M')
    min_diff = datetime.timedelta(days=365*1000)  # initialize with a big time difference
    closest_file = None

    # Iterate over all files in the directory
    for filename in os.listdir(directory):
        # Check if filename is a Templog
        if '__Templog.txt' in filename:
            # Extract timestamp from filename and convert to datetime object
            file_timestamp = filename[:16]
            file_datetime = datetime.datetime.strptime(file_timestamp, '%Y%m%d_%Hh_%Mm')

            # Calculate time difference
            diff = abs(date_time_obj - file_datetime)

            # If this file is closer, update min_diff and closest_file
            if diff < min_diff:
                min_diff = diff
                closest_file = filename

    if closest_file is None:
        print("Logfile not found")
    else:
        file_path = os.path.join(directory, closest_file)
        file_size = os.path.getsize(file_path)  # Get file size in bytes
        print(f"Closest logfile: {closest_file}, Size: {file_size} bytes")
    
    # Read the log file and parse the relevant data

    df_log = create_log_df(file_path)

    # Match the data with the profile dataframe
    query = f'SELECT * FROM "{profile}"'
    df = pd.read_sql_query(query, engine)
    df.drop(['TT_x', 'T1_x', 'T2_x', 'TH_x', 'restart_x', 'relock_x', 'Time_log_x', 'TT_y', 'T1_y', 'T2_y', 'TH_y', 'restart_y', 'relock_y', 'Time_log_y', 'TT', 'T1', 'T2', 'TH', 'restart', 'relock', 'Time_log', 'TAG_event', 'TAG_event_x', 'TAG_event_y', 'near_event', 'near_event_x', 'near_event_y'], axis=1, inplace=True, errors='ignore')
    # Convert the timestamps in both dataframes to datetime format
    df['timestamp'] = df['date-time']
    #df_log['timestamp_l'] = df_log['Time_log']

    df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y%m%d-%H%M%S%f')
    #df_log['timestamp_l'] = pd.to_datetime(df_log['timestamp_l'], format='%Y%m%d-%H%M%S')

    # Sort the dataframes by the timestamp
    df = df.sort_values('timestamp')
    df_log = df_log.sort_values('timestamp')

    # Use merge_asof to merge the two dataframes, finding the nearest match on the timestamp
    df_combined = pd.merge_asof(df, df_log, left_on='timestamp', right_on='timestamp', direction='backward')
    df_combined.drop('timestamp', axis=1, inplace=True)
    #df_combined.drop('timestamp', axis=1, inplace=True)

    #print(df_combined.head())

    # Update database
    df_combined.to_sql(profile, engine, if_exists='replace', index=False)
    print("Updated {}".format(profile))




Closest logfile: 20220512_17h_47m__Templog.txt, Size: 509203 bytes
Updated M181-227-1_CTD-060_00deg00S-027deg00W_20220512-1748
Closest logfile: 20220505_14h_01m__Templog.txt, Size: 821675 bytes
Updated M181-126-1_CTD-040_00deg00S-010deg00W_20220505-1401
Closest logfile: 20220504_07h_56m__Templog.txt, Size: 846382 bytes
Updated M181-107-1_CTD-036_00deg00S-007deg00W_20220504-0756
Closest logfile: 20220509_05h_43m__Templog.txt, Size: 717942 bytes
Updated M181-175-1_CTD-050_00deg00S-019deg00W_20220509-0543
Closest logfile: 20220516_03h_34m__Templog.txt, Size: 761965 bytes
Updated M181-267-1_CTD-070_00deg00S-035deg00W_20220516-0335
Closest logfile: 20220506_18h_22m__Templog.txt, Size: 772615 bytes
Updated M181-140-1_CTD-043_00deg00S-013deg00W_20220506-1822
Closest logfile: 20220508_00h_38m__Templog.txt, Size: 568372 bytes
Updated M181-160-1_CTD-047_00deg00S-016deg00W_20220508-0038
Closest logfile: 20220517_20h_24m__Templog.txt, Size: 627455 bytes
Updated M181-285-1_CTD-075_00deg00S-039deg00

In [188]:
restart_rows = df[df['relock'] == True]
print(len(restart_rows), restart_rows)

15                         TT     T1     T2     TH  relock  restart
timestamp                                                       
2022-05-18 21:07:34  27.97  27.75  28.19  27.81    True    False
2022-05-18 21:16:25  26.94  26.38  27.50  26.88    True    False
2022-05-18 21:18:47  25.91  25.19  26.63  25.69    True    False
2022-05-18 21:20:55  24.88  24.00  25.75  24.50    True    False
2022-05-18 21:23:03  23.84  22.88  24.81  23.38    True    False
2022-05-18 21:25:12  22.78  21.75  23.81  22.19    True    False
2022-05-18 21:27:21  21.75  20.69  22.81  21.06    True    False
2022-05-18 21:29:33  20.72  19.63  21.81  20.00    True    False
2022-05-18 21:46:14  20.16  17.81  22.50  19.44    True    False
2022-05-18 22:02:56  20.06  17.44  22.69  19.19    True    False
2022-05-18 22:19:38  20.00  17.19  22.81  19.13    True    False
2022-05-18 22:36:19  20.09  17.19  23.00  20.31    True    False
2022-05-18 22:53:01  20.19  17.38  23.00  19.69    True    False
2022-05-18 23:09:43  2

### Add Object id (per Profile)

In [ ]:
for profile in tqdm(table_names):
    query = f'SELECT * FROM "{profile}"'
    df = pd.read_sql_query(query, engine)
    
    df.drop(['obj_id'], axis=1, inplace=True, errors='ignore')
    sorted_df = df.sort_values(by='filename')
    print(sorted_df.head())
    sorted_fn_list = sorted_df['filename'].tolist()
    obj_ids = []
    id_cnt = 0
    for img in sorted_fn_list:
        curr_id = id_cnt
        obj_ids.append('obj_'+str(curr_id))
        id_cnt = id_cnt+1
    sorted_df['obj_id'] = obj_ids
    sorted_df.to_sql(profile, engine, if_exists='replace', index=False)
    print("Updated {}".format(profile))

    

### Calculate UMAP Embeddings for selected "handcrafted" features

In [20]:
import umap
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from sklearn.preprocessing import StandardScaler
import os

root_dir = '/media/veit/30781fe1-cea5-4503-ae00-1986beb935d2/Segmentation_results/M181/results_240328'

selected_features = [
       # 'pressure [dbar]', 
       # 'temperature', 
       # 'area', 
       # 'w', 
       # 'h', 
       'esd', 
       # 'interpolated_s', 
       # 'interpolated_o',
       # 'interpolated_t', 
       # 'interpolated_chl', 
       'object_area_exc', 
       'object_area_rprops', 
       'object_%area',
       'object_major_axis_len', 
       'object_minor_axis_len', 
       'object_centroid_y', 
       'object_centroid_x', 
       'object_convex_area',
       'object_min_intensity', 
       'object_max_intensity', 
       'object_mean_intensity', 
       'object_int_density', 
       'object_perimeter',
       'object_elongation', 
       'object_range', 
       'object_perim_area_excl', 
       'object_perim_major', 
       'object_circularity_area_excl', 
       'object_angle',
       'object_boundbox_area', 
       'object_eccentricity', 
       'object_equivalent_diameter',
       'object_euler_nr', 
       'object_extent', 
       'object_local_centroid_col', 
       'object_local_centroid_row',
       'object_solidity', 
       'TAG_event', 
       'part_based_filter'
]

all_dfs = []  # list to store all the dataframes

for profile in table_names:
       query = f'SELECT * FROM "{profile}"'
       df = pd.read_sql_query(query, engine)
       df_selected = df[selected_features]
       all_dfs.append(df_selected)  # append the selected dataframe to the list

# combine all dataframes
df_selected_comb = pd.concat(all_dfs, ignore_index=True)

# #fit the UMAP model to the first profile
# query = f'SELECT * FROM "{table_names[0]}"'
# df = pd.read_sql_query(query, engine)
# df_selected = df[selected_features]
#print(df_selected.head())
# Same for a StandardScaler instance
scaler = StandardScaler().fit(df_selected_comb)
df_selected_scaled = scaler.transform(df_selected_comb)
#print(df_selected_scaled)
reducer = umap.UMAP(metric='cosine', verbose=True).fit(df_selected_scaled)

# Save the fitted scaler and reducer to disk
scaler_path = os.path.join(root_dir, 'standard_scaler.pkl')
with open(scaler_path, 'wb') as f:
    pickle.dump(scaler, f)
print(f'StandardScaler saved to {scaler_path}')

reducer_path = os.path.join(root_dir, 'umap_reducer.pkl')
with open(reducer_path, 'wb') as f:
    pickle.dump(reducer, f)
print(f'UMAP reducer saved to {reducer_path}')

for profile in table_names:
       query = f'SELECT * FROM "{profile}"'
       df = pd.read_sql_query(query, engine)
       df_selected = df[selected_features]
       
       # transform the scaler to the selected features
       df_selected_scaled = scaler.transform(df_selected)

       # Then transform the UMAP model with the scaled data
       embedding = reducer.transform(df_selected_scaled) 

       #plot embedding 
       plt.scatter(embedding[:, 0], embedding[:, 1])
       plt.gca().set_aspect('equal', 'datalim')
       plt.title('UMAP projection of the selected features', fontsize=18)
       plt.show()
       
       #add embedding to database
       df.drop(['umap_x', 'umap_y'], axis=1, inplace=True, errors='ignore')
       df['umap_x']=embedding[:, 0]
       df['umap_y']=embedding[:, 1]
       df.to_sql(profile, engine, if_exists='replace', index=False)
       print("Updated {}".format(profile))

       #save embedding to pickle file
       # profile = profile.replace('deg', '°')  
       # dest = os.path.join(root_dir, profile, 'handcrafted_embedding.pkl')
       # with open(dest, 'wb') as f:
       #        pickle.dump(embedding, f)
       # print(f'UMAP embedding saved to {dest}')

/home/veit/anaconda3/envs/pisco-dev/lib/python3.10/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


UMAP(angular_rp_forest=True, metric='cosine', verbose=True)
Tue Mar 25 14:52:25 2025 Construct fuzzy simplicial set
Tue Mar 25 14:52:25 2025 Finding Nearest Neighbors
Tue Mar 25 14:52:25 2025 Building RP forest with 64 trees
Tue Mar 25 14:52:36 2025 NN descent for 21 iterations
	 1  /  21
	 2  /  21
	Stopping threshold met -- exiting after 2 iterations
Tue Mar 25 14:53:13 2025 Finished Nearest Neighbor Search
Tue Mar 25 14:53:18 2025 Construct embedding


Epochs completed:   2%| ▏          3/200 [00:00]

	completed  0  /  200 epochs


Epochs completed:  11%| █          22/200 [00:03]

...

Epochs completed:  91%| █████████  182/200 [00:25]

	completed  180  /  200 epochs


Epochs completed: 100%| ██████████ 200/200 [00:28]


Tue Mar 25 15:10:29 2025 Finished embedding


FileNotFoundError: [Errno 2] No such file or directory: '/media/veit/30781fe1-cea5-4503-ae00-1986beb935d2/Segmentation_results/M181/results_240328/standard_scaler.pkl'

In [21]:
# Save the fitted scaler and reducer to disk
scaler_path = os.path.join(root_dir, 'standard_scaler.pkl')
with open(scaler_path, 'wb') as f:
    pickle.dump(scaler, f)
print(f'StandardScaler saved to {scaler_path}')

reducer_path = os.path.join(root_dir, 'umap_reducer.pkl')
with open(reducer_path, 'wb') as f:
    pickle.dump(reducer, f)
print(f'UMAP reducer saved to {reducer_path}')

StandardScaler saved to /media/veit/30781fe1-cea5-4503-ae00-1986beb935d2/Segmentation_results/M181/results_240328/standard_scaler.pkl
Tue Mar 25 15:13:00 2025 Worst tree score: 0.69144161
Tue Mar 25 15:13:00 2025 Mean tree score: 0.69391879
Tue Mar 25 15:13:00 2025 Best tree score: 0.69594535
Tue Mar 25 15:13:05 2025 Forward diversification reduced edges from 26747790 to 12961216
Tue Mar 25 15:13:06 2025 Reverse diversification reduced edges from 12961216 to 12961216
Tue Mar 25 15:13:07 2025 Degree pruning reduced edges from 13973406 to 13973406
Tue Mar 25 15:13:07 2025 Resorting data and graph based on tree order
Tue Mar 25 15:13:07 2025 Building and compiling search function
UMAP reducer saved to /media/veit/30781fe1-cea5-4503-ae00-1986beb935d2/Segmentation_results/M181/results_240328/umap_reducer.pkl


### add embedding to df afterwards

In [19]:
import pickle

for profile in table_names:
       query = f'SELECT * FROM "{profile}"'
       df = pd.read_sql_query(query, engine)
       features_loc = os.path.dirname(os.path.dirname(df['overview_path'].iloc[0])) + '/handcrafted_embedding.pkl'
       with open(features_loc, 'rb') as f:
              embedding = pickle.load(f)
       df['umap_x']=embedding[:, 0]
       df['umap_y']=embedding[:, 1]
       df.to_sql(profile, engine, if_exists='replace', index=False)
       print("Updated {}".format(profile))

Updated M181-245-1_CTD-065_00deg00S-031deg00W_20220514-0907
Updated M181-200-1_CTD-055_00deg00S-023deg00W_20220511-0023
Updated M181-227-1_CTD-060_00deg00S-027deg00W_20220512-1748
Updated M181-126-1_CTD-040_00deg00S-010deg00W_20220505-1401
Updated M181-107-1_CTD-036_00deg00S-007deg00W_20220504-0756
Updated M181-175-1_CTD-050_00deg00S-019deg00W_20220509-0543
Updated M181-267-1_CTD-070_00deg00S-035deg00W_20220516-0335
Updated M181-140-1_CTD-043_00deg00S-013deg00W_20220506-1822
Updated M181-160-1_CTD-047_00deg00S-016deg00W_20220508-0038
Updated M181-285-1_CTD-075_00deg00S-039deg00W_20220517-2024
Updated M181-295-1_CTD-080_00deg00S-041deg00W_20220518-2053


### add prediction of schlieren images by filtering based on particle counts per image

In [6]:
for profile in table_names:
    query = f'SELECT * FROM "{profile}"'
    df = pd.read_sql_query(query, engine)
    df.drop(['part_based_filter'], axis=1, inplace=True, errors='ignore')

    df_unique = df[['date-time', 'pressure [dbar]', 'depth [m]', 'img_id','temperature','overview_path','interpolated_s','interpolated_t','interpolated_o','interpolated_chl','interpolated_z_factor','restart','relock','TAG_event']].drop_duplicates()
    df_count = df.groupby('date-time').size().reset_index(name='count')
    df_unique = df_unique.merge(df_count, on='date-time', how='left')
    df_unique = df_unique.sort_values('pressure [dbar]')


    # Filter the data
    df_unique['part_based_filter'] = df_unique['count'].apply(lambda x: 0 if x < df_unique['count'].std() else 1)
    #print(df_unique.head())
    filtered_df = df_unique[df_unique['part_based_filter'] == 0]

    # Merge 'df_unique' back to 'df' to create 'part_based_filter' column in 'df'
    df = df.merge(df_unique[['date-time', 'part_based_filter']], on='date-time', how='left')
    df.to_sql(profile, engine, if_exists='replace', index=False)
    print(df['part_based_filter'].value_counts())
    print("Updated {}".format(profile))
    #print(df.head())

Updated M181-160-1_CTD-047_00deg00S-016deg00W_20220508-0038
Updated M181-245-1_CTD-065_00deg00S-031deg00W_20220514-0907
Updated M181-200-1_CTD-055_00deg00S-023deg00W_20220511-0023
Updated M181-227-1_CTD-060_00deg00S-027deg00W_20220512-1748
Updated M181-126-1_CTD-040_00deg00S-010deg00W_20220505-1401
Updated M181-107-1_CTD-036_00deg00S-007deg00W_20220504-0756
Updated M181-175-1_CTD-050_00deg00S-019deg00W_20220509-0543
Updated M181-267-1_CTD-070_00deg00S-035deg00W_20220516-0335
Updated M181-140-1_CTD-043_00deg00S-013deg00W_20220506-1822
Updated M181-285-1_CTD-075_00deg00S-039deg00W_20220517-2024
Updated M181-295-1_CTD-080_00deg00S-041deg00W_20220518-2053


In [8]:
print(df['part_based_filter'].value_counts())

           date-time  pressure [dbar]  depth [m]  temperature  index  img_id  \
0  20220506-18475929            -0.08      -0.08        32.27   1005       1   
1  20220506-18475929            -0.08      -0.08        32.27   1012       1   
2  20220506-18475929            -0.08      -0.08        32.27    103       1   
3  20220506-18475929            -0.08      -0.08        32.27   1040       1   
4  20220506-18475929            -0.08      -0.08        32.27    105       1   

                                       filename    area       x       y  ...  \
0  20220506-18475929_000.992bar_32.27C_1005.png  9945.0  2102.5   266.5  ...   
1  20220506-18475929_000.992bar_32.27C_1012.png   459.5   508.5   202.5  ...   
2   20220506-18475929_000.992bar_32.27C_103.png   508.5  2476.5  2105.5  ...   
3  20220506-18475929_000.992bar_32.27C_1040.png   690.0   362.0   136.0  ...   
4   20220506-18475929_000.992bar_32.27C_105.png   440.0   445.5  2097.5  ...   

      TT     T1     T2     TH  relock 

### Export images to a directory and zip it

In [5]:
import os
import shutil
from PIL import Image

# create a new directory for the images
os.makedirs('/media/plankton/30781fe1-cea5-4503-ae00-1986beb935d2/exported_images', exist_ok=True)
df_all = pd.DataFrame()

for profile in table_names:
    query = f'SELECT * FROM "{profile}"'
    df = pd.read_sql_query(query, engine)
    # go through the DataFrame
    df_filtered = df[(df['TAG_event'] == 0) & (df['part_based_filter'] == 0)]
    # for idx, row in df_filtered.iterrows():
    #     # read the image
    #     img = Image.open(row['full_path'])
    #     # save to the new location
    #     img.save(os.path.join('/media/plankton/30781fe1-cea5-4503-ae00-1986beb935d2/exported_images', os.path.basename(row['full_path'])))
    print("Loaded images from {}".format(profile))
    df_all = pd.concat([df_all, df_filtered])

# save df_all to a csv file
df_all.to_csv('/media/plankton/30781fe1-cea5-4503-ae00-1986beb935d2/exported_images.csv')

# make a zip file
#shutil.make_archive('/media/plankton/30781fe1-cea5-4503-ae00-1986beb935d2/exported_images', 'zip', '/media/plankton/30781fe1-cea5-4503-ae00-1986beb935d2/exported_images')

Loaded images from M181-175-1_CTD-050_00deg00S-019deg00W_20220509-0543
Loaded images from M181-267-1_CTD-070_00deg00S-035deg00W_20220516-0335
Loaded images from M181-140-1_CTD-043_00deg00S-013deg00W_20220506-1822
Loaded images from M181-160-1_CTD-047_00deg00S-016deg00W_20220508-0038
Loaded images from M181-245-1_CTD-065_00deg00S-031deg00W_20220514-0907
Loaded images from M181-200-1_CTD-055_00deg00S-023deg00W_20220511-0023
Loaded images from M181-227-1_CTD-060_00deg00S-027deg00W_20220512-1748
Loaded images from M181-126-1_CTD-040_00deg00S-010deg00W_20220505-1401
Loaded images from M181-285-1_CTD-075_00deg00S-039deg00W_20220517-2024
Loaded images from M181-295-1_CTD-080_00deg00S-041deg00W_20220518-2053
Loaded images from M181-107-1_CTD-036_00deg00S-007deg00W_20220504-0756


### Extract crops from raw images 

In [5]:
import os
import shutil
from PIL import Image

raw_root = '/mnt/m181'

# scale the coordinates and the size of the crop
scale_factor = 5120 / 2560  # target_resolution / original_resolution

for profile in table_names:
    image_name_old = ''
    query = f'SELECT * FROM "{profile}"'
    df = pd.read_sql_query(query, engine)
    # go through the DataFrame
    df_filtered = df[(df['TAG_event'] == 0) & (df['part_based_filter'] == 0)]
    for idx, row in df_filtered.iterrows():
        # read the image
        img_name = os.path.basename(row['overview_path'])
        if img_name != image_name_old:            
            raw_img_path = os.path.join(raw_root, profile, 'PNG', img_name)
            img = Image.open(raw_img_path)
            image_name_old = img_name
        
        x = row['x']
        y = row['y']
        w = row['w']
        h = row['h']

        
        x = x * scale_factor
        y = y * scale_factor
        w = w * scale_factor
        h = h * scale_factor

        # calculate the top left corner of the crop
        x1 = x - w / 2
        y1 = y - h / 2

        # calculate the bottom right corner of the crop
        x2 = x + w / 2
        y2 = y + h / 2

        # crop the image
        cropped_image = img.crop((x1, y1, x2, y2))

        # save the cropped image
        cropped_image.save(os.path.join('/media/plankton/30781fe1-cea5-4503-ae00-1986beb935d2/exported_images_raw', os.path.basename(row['full_path'])))
    print("Loaded images from {}".format(profile))
shutil.make_archive('/media/plankton/30781fe1-cea5-4503-ae00-1986beb935d2/exported_images_raw', 'zip', '/media/plankton/30781fe1-cea5-4503-ae00-1986beb935d2/exported_images_raw')

Loaded images from M181-175-1_CTD-050_00deg00S-019deg00W_20220509-0543
Loaded images from M181-267-1_CTD-070_00deg00S-035deg00W_20220516-0335
Loaded images from M181-140-1_CTD-043_00deg00S-013deg00W_20220506-1822
Loaded images from M181-160-1_CTD-047_00deg00S-016deg00W_20220508-0038
Loaded images from M181-245-1_CTD-065_00deg00S-031deg00W_20220514-0907
Loaded images from M181-200-1_CTD-055_00deg00S-023deg00W_20220511-0023
Loaded images from M181-227-1_CTD-060_00deg00S-027deg00W_20220512-1748
Loaded images from M181-126-1_CTD-040_00deg00S-010deg00W_20220505-1401
Loaded images from M181-285-1_CTD-075_00deg00S-039deg00W_20220517-2024
Loaded images from M181-295-1_CTD-080_00deg00S-041deg00W_20220518-2053
Loaded images from M181-107-1_CTD-036_00deg00S-007deg00W_20220504-0756


### Export binary masks to zip

In [8]:
import os
import shutil
from PIL import Image

# create a new directory for the images
os.makedirs('/media/plankton/30781fe1-cea5-4503-ae00-1986beb935d2/exported_images_masks', exist_ok=True)
df_all = pd.DataFrame()

for profile in table_names:
    query = f'SELECT * FROM "{profile}"'
    df = pd.read_sql_query(query, engine)
    # go through the DataFrame
    df_filtered = df[(df['TAG_event'] == 0) & (df['part_based_filter'] == 0)]
    for idx, row in df_filtered.iterrows():
        mask_name = os.path.basename(row['full_path'])
        mask_name = mask_name[:-4] + '_mask.png'
        mask_path = os.path.join(os.path.dirname(os.path.dirname(row['full_path'])), 'Masks', mask_name)
        # read the image
        img = Image.open(mask_path)
        # save to the new location
        img.save(os.path.join('/media/plankton/30781fe1-cea5-4503-ae00-1986beb935d2/exported_images_masks', mask_name))
    print("Loaded images from {}".format(profile))
    df_all = pd.concat([df_all, df_filtered])

# save df_all to a csv file
df_all.to_csv('/media/plankton/30781fe1-cea5-4503-ae00-1986beb935d2/exported_images.csv')

# make a zip file
shutil.make_archive('/media/plankton/30781fe1-cea5-4503-ae00-1986beb935d2/exported_images_masks', 'zip', '/media/plankton/30781fe1-cea5-4503-ae00-1986beb935d2/exported_images_masks')

Loaded images from M181-175-1_CTD-050_00deg00S-019deg00W_20220509-0543
Loaded images from M181-267-1_CTD-070_00deg00S-035deg00W_20220516-0335
Loaded images from M181-140-1_CTD-043_00deg00S-013deg00W_20220506-1822
Loaded images from M181-160-1_CTD-047_00deg00S-016deg00W_20220508-0038
Loaded images from M181-245-1_CTD-065_00deg00S-031deg00W_20220514-0907
Loaded images from M181-200-1_CTD-055_00deg00S-023deg00W_20220511-0023
Loaded images from M181-227-1_CTD-060_00deg00S-027deg00W_20220512-1748
Loaded images from M181-126-1_CTD-040_00deg00S-010deg00W_20220505-1401
Loaded images from M181-285-1_CTD-075_00deg00S-039deg00W_20220517-2024
Loaded images from M181-295-1_CTD-080_00deg00S-041deg00W_20220518-2053
Loaded images from M181-107-1_CTD-036_00deg00S-007deg00W_20220504-0756


'/media/plankton/30781fe1-cea5-4503-ae00-1986beb935d2/exported_images_masks.zip'

In [13]:
engine.dispose()